In [16]:
import os
# get a token: https://huggingface.co/docs/api-inference/quicktour#get-your-api-token

from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()


In [17]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [163]:
from langchain_core.prompts import ChatPromptTemplate
template = """
Based on the table schema below, write sql query that would answer the user's question and don't give any description only the sql query:
{schema}

Question:{question}
SQL Query:
"""

prompt = ChatPromptTemplate.from_template(template)

In [164]:
prompt.format(schema="my schema",question="how many proteins are available?")

"Human: \nBased on the table schema below, write sql query that would answer the user's question and don't give any description only the sql query:\nmy schema\n\nQuestion:how many proteins are available?\nSQL Query:\n"

In [165]:
from langchain_community.utilities import SQLDatabase

In [166]:
db_url = 'mysql+mysqlconnector://newuser@localhost:3306/protein_db_small'
db = SQLDatabase.from_uri(db_url)

In [167]:
def get_schema(_):
    return db.get_table_info()

In [168]:
get_schema(None)

"\nCREATE TABLE active_sites (\n\tsite_id INTEGER NOT NULL AUTO_INCREMENT, \n\tprotein_id INTEGER, \n\tdescription TEXT, \n\tposition VARCHAR(50), \n\tPRIMARY KEY (site_id), \n\tCONSTRAINT active_sites_ibfk_1 FOREIGN KEY(protein_id) REFERENCES proteins (protein_id) ON DELETE CASCADE\n)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci\n\n/*\n3 rows from active_sites table:\nsite_id\tprotein_id\tdescription\tposition\n1\t1\tProton donor\t133\n2\t1\tProton acceptor\t235\n3\t3\tECO:0000255|HAMAP-Rule:MF_01013\t12\n*/\n\n\nCREATE TABLE allergenic_properties (\n\tproperty_id INTEGER NOT NULL AUTO_INCREMENT, \n\tprotein_id INTEGER, \n\tdescription TEXT, \n\tPRIMARY KEY (property_id), \n\tCONSTRAINT allergenic_properties_ibfk_1 FOREIGN KEY(protein_id) REFERENCES proteins (protein_id) ON DELETE CASCADE\n)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci\n\n/*\n3 rows from allergenic_properties table:\nproperty_id\tprotein_id\tdescription\n\n*/\n\n\nCREATE TABLE bin

In [169]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFaceEndpoint
from typing import Optional


In [ ]:
model_path = r"E:\LLM\models\unsloth\Llama-3.2-3B-Instruct-GGUF\Llama-3.2-3B-Instruct-Q8_0.gguf" #use model here

In [171]:
from ctransformers import AutoModelForCausalLM
from langchain_community.llms import LlamaCpp
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import os

def setup_local_llm(
    model_path: str,
    temperature: float = 0.7,
    max_tokens: int = 2000,
    n_ctx: int = 5000,
    n_gpu_layers: int = 1,
    verbose: bool = True
):
    """
    Set up a local LLM using a GGUF model file.
    
    Args:
        model_path: Path to the .gguf model file
        temperature: Model temperature (higher = more creative)
        max_tokens: Maximum tokens in the response
        n_ctx: Size of the context window
        n_gpu_layers: Number of layers to offload to GPU
        verbose: Whether to print debug information
    """
    
    # Initialize the LlamaCpp model
    llm = LlamaCpp(
        model_path=model_path,
        temperature=temperature,
        max_tokens=max_tokens,
        n_ctx=n_ctx,
        n_gpu_layers=n_gpu_layers,
        verbose=verbose,
    )
    
    return llm
# Path to your local GGUF model
    
# Load the model


In [172]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

llmin = HuggingFaceEndpoint(
    repo_id=repo_id,
    max_length=256,
    temperature=0.5,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
)

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


In [173]:

llmout = setup_local_llm(
        model_path=model_path,
        temperature=0.7,
        n_gpu_layers=1  # Set to 0 if you don't have a GPU
    )


llama_model_loader: loaded meta data with 30 key-value pairs and 255 tensors from E:\LLM\models\unsloth\Llama-3.2-3B-Instruct-GGUF\Llama-3.2-3B-Instruct-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 3B Instruct
llama_model_loader: - kv   3:                       general.organization str              = Meta Llama
llama_model_loader: - kv   4:                           general.finetune str              = Instruct
llama_model_loader: - kv   5:                           general.basename str              = Llama-3.2
llama_model_loader: - kv   6:                         general.size_label str              = 3B
l

In [174]:
sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llmin.bind(stop=["\nSQL Result:"])
    | StrOutputParser()

)


In [179]:
template = """
You are a precise protein database expert.
Provide a clear, concise response that:
1. Directly answers the question
2. Only includes relevant protein information
3. Uses scientific terminology appropriately
4. Avoids speculation or unnecessary elaboration
5. Avoid giving SQL query at response
Based on the table schema below, question, sql query and sql response, write a natural language response:

Table Schema:
{schema}

Original Question: {question}
SQL Query Used: {query}
Database Result: {response}

Response:"""

prompt = ChatPromptTemplate.from_template(template)

In [176]:
def run_query(query):
    return db.run(query)

In [177]:
full_chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(
        schema=get_schema,
        response = lambda variables: run_query(variables["query"])
    )
    | prompt
    | llmout
)

In [178]:
qn = "what is proteins name of the sequence MRSLAILTTLLAGHAFAYPKPAPQSVNRRDWPSINEFLSELAKVMPIGDTITAACDLISDGEDAAASLFGISETENDPCGDVTVLFARGTCDPGNVGVLVGPWFFDSLQTALGSRTLGVKGVPYPASVQDFLSGSVQNGINMANQIKSVLQSCPNTKLVLGGYSQGSMVVHNAASNLDAATMSKISAVVLFGDPYYGKPVANFDAAKTLVVCHDGDNICQGGDIILLPHLTYAEDADTAAAFVVPLVS"
sql_chain.invoke({"question":qn})
print(full_chain.invoke({"question":qn}))

llama_perf_context_print:        load time =   94592.31 ms
llama_perf_context_print: prompt eval time =       0.00 ms /  3446 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   801 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  185375.16 ms /  4247 tokens


 The protein entry name associated with the sequence 'MRSLAILTTLLAGHAFAYPKPAPQSVNRRDWPSINEFLSELAKVMPIGDTITAACDLISDGEDAAASLFGISETENDPCGDVTVLFARGTCDPGNVGVLVGPWFFDSLQTALGSRTLGVKGVPYPASVQDFLSGSVQNGINMANQIKSVLQSCPNTKLVLGGYSQGSMVVHNAASNLDAATMSKISAVVLFGDPYYGKPVANFDAAKTLVVCHDGDNICQGGDIILLPHLTYAEDADTAAAFVVPLVS'

The protein entry name associated with the sequence 'MRSLAILTTLLAGHAFAYPKPAPQSVNRRDWPSINEFLSELAKVMPIGDTITAACDLISDGEDAAASLFGISETENDPCGDVTVLFARGTCDPGNVGVLVGPWFFDSLQTALGSRTLGVKGVPYPASVQDFLSGSVQNGINMANQIKSVLQSCPNTKLVLGGYSQGSMVVHNAASNLDAATMSKISAVVLFGDPYYGKPVANFDAAKTLVVCHDGDNICQGGDIILLPHLTYAEDADTAAAFVVPLVS'

is Cuti1 HypoJ (cuti1_hypoJ) and its entry name is MENC_SALTY. 

This protein was previously identified as being associated with a salt-dependent membrane transport system in the pathogen Pseudomonas aeruginosa.

The sequence provided corresponds to the protein entry of Cuti1 HypoJ, which has an accession number of A3NE93.

Therefore, the response is that the protein entry name for the gi